# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved.
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server.

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
from collections import defaultdict
words = []
with open('words_250000_train.txt','r') as file:
    for line in file:
        words.append(line.strip())

def generate_ngram_counts(word_list):
    ngram_counts = defaultdict(lambda: defaultdict(int))
    left_bigram_counts = defaultdict(lambda: defaultdict(int))
    right_bigram_counts = defaultdict(lambda: defaultdict(int))

    for word in word_list:
        length = len(word)

        for i in range(length - 2):
            left, middle, right = word[i], word[i+1], word[i+2]

            context = left + right
            ngram_counts[context][middle] += 1

        for i in range(length - 1):
            left, next_letter = word[i], word[i + 1]
            left_bigram_counts[left][next_letter] += 1

        for i in range(1, length):
            right, prev_letter = word[i], word[i - 1]
            right_bigram_counts[right][prev_letter] += 1

    def normalize_to_vector(counts):
        normalized_vectors = {}
        for context, letter_counts in counts.items():
            total_count = sum(letter_counts.values())
            vector = [0] * 26

            for letter, count in letter_counts.items():
                index = ord(letter) - ord('a')
                if 0 <= index < 26:
                    vector[index] = count / total_count
            normalized_vectors[context] = vector

        return normalized_vectors

    normalized_ngram_counts = normalize_to_vector(ngram_counts)
    normalized_left_bigram_counts = normalize_to_vector(left_bigram_counts)
    normalized_right_bigram_counts = normalize_to_vector(right_bigram_counts)

    return normalized_ngram_counts, normalized_left_bigram_counts, normalized_right_bigram_counts

ngram_counts, left_bigram_counts, right_bigram_counts = generate_ngram_counts(words)


In [3]:
import torch
import random
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from transformers import BertTokenizer, BertForMaskedLM
import torch.nn.functional as F

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased')

class BertHangman(nn.Module):
  def __init__(self, base_model, vocab_size = 26, hidden_size = 328):
    super(BertHangman,self).__init__()
    self.model = base_model.bert

    for param in self.model.parameters():
        param.requires_grad = False

    for layer in self.model.encoder.layer[10:]:
        for param in layer.parameters():
            param.requires_grad = True


    self.dropout = nn.Dropout(0.2)
    self.fc = nn.Linear(768+vocab_size, hidden_size)
    self.relu = nn.ReLU()
    self.output = nn.Linear(hidden_size, vocab_size)


  def forward(self, input_ids=None, attention_mask=None, guessed_letters=None):

    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
    sequence_output = outputs.last_hidden_state
    sequence_output = self.dropout(sequence_output)


    sequence_output = torch.cat((sequence_output, guessed_letters.unsqueeze(1).repeat(1, sequence_output.shape[1], 1)), dim=2)

    logits = self.fc(sequence_output)
    logits = self.relu(logits)
    logits = self.output(logits)


    logits[guessed_letters.unsqueeze(1).repeat(1, sequence_output.shape[1], 1) == 1] = -float("inf")

    return logits

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
char_to_index = {chr(i + ord('a')): i for i in range(26)}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

In [12]:
hangman_model = BertHangman(bert_model, vocab_size = 26).to(device)
checkpoint = torch.load('bert_ce_complete_model_epoch_100.pth', map_location=device)
hangman_model.load_state_dict(checkpoint['model_state_dict'])
hangman_model.eval()

<ipython-input-12-42c4082bd3fe>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('bert_ce_complete_model_epoch_100.pth', map_location=device)


BertHangman(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [17]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()

        self.current_dictionary = []

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        hangman_model.eval()
        # print('WORD',word)
        cleaned_word = word.replace(" ", "")
        guessed = set(self.guessed_letters)
        guessed_list = self.guessed_letters
        word_list = list(cleaned_word)
        word_set = set(cleaned_word)
        label_letters = [-100 for char in word_list]
        letters_to_be_masked = word_set - guessed

        tokenized_word = bert_tokenizer(
          " ".join(list(cleaned_word)),
          return_tensors="pt",
          truncation=True,
        )

        input_ids = tokenized_word.input_ids.clone().squeeze(0)
        attention_mask = tokenized_word.attention_mask.squeeze(0)
        guessed_letters = torch.zeros(26, dtype=torch.int64)
        for i in range(len(word_list)):
          if cleaned_word[i] == '_':
            input_ids[i+1] = bert_tokenizer.mask_token_id
            label_letters[i] = 0


        for i in range(len(guessed_list)):
            guessed_letters[ord(guessed_list[i]) - ord('a')] = 1
        logits = hangman_model(torch.unsqueeze(input_ids,0).to(device), torch.unsqueeze(attention_mask,0).to(device), torch.unsqueeze(guessed_letters,0).to(device))
        logits = F.softmax(logits, dim=-1)

        logits = logits.squeeze()[1:-1,:]
        label_mask = [x != -100 for x in label_letters]

        ngram_probabilities = torch.zeros_like(logits)

        label_mask = [x != -100 for x in label_letters]

        missing_count = sum(label_mask)

        if len(cleaned_word) > 3 and missing_count <= 3:
           for i in range(len(cleaned_word)):
                left = None
                right = None
                if i!=0 and i!=len(cleaned_word)-1 and label_letters[i] != -100:
                  if label_letters[i-1] == -100:
                    left = cleaned_word[i-1]
                  if label_letters[i+1] == -100:
                    right = cleaned_word[i+1]
                  if left is not None and right is not None:
                    context = left+right
                    if context in ngram_counts:
                      ngram_probabilities[i] = torch.tensor(ngram_counts[context])
                  elif left is None and right is None:
                    pass
                  elif left is None:
                    if right in right_bigram_counts:
                      ngram_probabilities[i] = torch.tensor(right_bigram_counts[right])
                  elif right is None:
                    if left in left_bigram_counts:
                      ngram_probabilities[i] = torch.tensor(left_bigram_counts[left])
                elif i==0 and label_letters[i] != -100:
                  if label_letters[i+1] == -100:
                      right = cleaned_word[i+1]
                  if right is not None and right in right_bigram_counts:
                      ngram_probabilities[i] = torch.tensor(right_bigram_counts[right])
                elif i==len(word)-1 and label_letters[i] != -100:
                  if label_letters[i-1] == -100:
                      left = cleaned_word[i-1]
                  if left is not None and left in left_bigram_counts:
                      ngram_probabilities[i] = torch.tensor(left_bigram_counts[left])
        alpha = 0.9
        logits = alpha *logits +  (1- alpha) * ngram_probabilities

        contributing_logits = logits[label_mask]

        max_prob_for_each_sequence = torch.max(contributing_logits, dim=-1).values
        max_prob_char_for_each_sequence = torch.argmax(contributing_logits, dim=-1)
        max_prob_index_out_of_max_prob_from_each_sequence = torch.argmax(max_prob_for_each_sequence)
        max_prob_char = max_prob_char_for_each_sequence[max_prob_index_out_of_max_prob_from_each_sequence]

        guess_letter = chr(ord('a')+max_prob_char)

        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                try:
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object.
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [18]:
api = HangmanAPI(access_token="3360fd537cec372abc0ddf538c01ce", timeout=2000)

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [19]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print(total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes)

Successfully start a new game! Game ID: 8029f141d5da. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: i
Sever response: {'game_id': '8029f141d5da', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ i _ _ i _ _ _ _ _ _ _ '}
Guessing letter: s
Sever response: {'game_id': '8029f141d5da', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ i _ _ i _ _ _ _ _ _ _ '}
Guessing letter: e
Sever response: {'game_id': '8029f141d5da', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ i _ _ i _ _ _ _ _ _ _ '}
Guessing letter: d
Sever response: {'game_id': '8029f141d5da', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ i _ _ i _ _ _ _ _ _ _ '}
Guessing letter: y
Sever response: {'game_id': '8029f141d5da', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ i _ _ i _ _ _ _ _ _ _ '}
Guessing letter: n
Sever response: {'game_id': '8029f141d5da', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ i _ _ i _ _ _ _ _ _ n '}
Guessing letter: o
Sever response: {'game_id': '8

In [20]:
for i in range(100):
  api.start_game(practice=1,verbose=True)
  [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
  practice_success_rate = (total_practice_successes-854) / (total_practice_runs-1752)
  print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs-1752, practice_success_rate))

Successfully start a new game! Game ID: c38e2c60d10b. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: i
Sever response: {'game_id': 'c38e2c60d10b', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ i _ _ _ _ _ _ '}
Guessing letter: s
Sever response: {'game_id': 'c38e2c60d10b', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ i _ _ _ _ _ _ '}
Guessing letter: e
Sever response: {'game_id': 'c38e2c60d10b', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e _ _ i _ _ _ _ _ _ '}
Guessing letter: y
Sever response: {'game_id': 'c38e2c60d10b', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ e _ _ i _ _ _ _ _ _ '}
Guessing letter: r
Sever response: {'game_id': 'c38e2c60d10b', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ e r _ i _ _ r _ _ _ '}
Guessing letter: o
Sever response: {'game_id': 'c38e2c60d10b', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ e r _ i _ _ r o o _ '}
Guessing letter: f
Sever response: {'game_id': 'c38e2c60d10b', 

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [21]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)

    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Playing  1  th game
Playing  2  th game
Playing  3  th game
Playing  4  th game
Playing  5  th game
Playing  6  th game
Playing  7  th game
Playing  8  th game
Playing  9  th game
Playing  10  th game
Playing  11  th game
Playing  12  th game
Playing  13  th game
Playing  14  th game
Playing  15  th game
Playing  16  th game
Playing  17  th game
Playing  18  th game
Playing  19  th game
Playing  20  th game
Playing  21  th game
Playing  22  th game
Playing  23  th game
Playing  24  th game
Playing  25  th game
Playing  26  th game
Playing  27  th game
Playing  28  th game
Playing  29  th game
Playing  30  th game
Playing  31  th game
Playing  32  th game
Playing  33  th game
Playing  34  th game
Playing  35  th game
Playing  36  th game
Playing  37  th game
Playing  38  th game
Playing  39  th game
Playing  40  th game
Playing  41  th game
Playing  42  th game
Playing  43  th game
Playing  44  th game
Playing  45  th game
Playing  46  th game
Playing  47  th game
Pl

HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [22]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.592
